# Deep Learning in Medicine
## BMSC-GA 4493, BMIN-GA 3007
## Homework 3: RNNs

Note 1: If you don't know how to run jupyter on the Prince cluster, here is another step-by-step guide here: 
<a href='https://docs.google.com/document/d/1HIdtzqJ6-RpsV0z2Gf5iXphNBTRca1kHZPlyqFxKpWs/edit?usp=sharing'> **Running Jupyter on the Cluster **</a>

Note 2: If you need to write mathematical terms, you can type your answeres in a Markdown Cell via LaTex
See: <a href="https://stackoverflow.com/questions/13208286/how-to-write-latex-in-ipython-notebook">here</a> if you have issues. To see basic LaTex notation see: <a href="https://en.wikibooks.org/wiki/LaTeX/Mathematics">here</a>.


Submission instruction: Upload and Submit your final jupyter notebook file in newclasses.nyu.edu

**Submission deadline: Thursday May 16th 2019 5pm.**



# Question 1: Literature Review for Sequence Classification: DeepBind (Total points 20 + 10 Bonus points)

Read this paper:

#### Babak Alipanahi1, Andrew Delong, Matthew T Weirauch & Brendan J Frey, *"Predicting the sequence specificities of DNA- and RNA-binding proteins by deep learning"* Nature Biotechnology, 2015  https://www.nature.com/articles/nbt.3300.pdf

We are interested in understanding the task, the methods that is proposed in this publication, technical aspects of the implementation, and possible future work.

**1.1) (5 points)** After you read the full article, go back to section **Training DeepBind and scoring sequences**. How do the authors define the **binding score, f(s)** on a given sequence? Write the formula here, and describe in your own words, in a few sentences, how the score is computed for each sequence (s)?


**1.2) (5 points)** What is the loss function that they are optimizing?

**1.3) (5 points)** What is the evaluation criteria based on which the authors do their cross validation? (Hint: Check Figure 2).

**1.4) (5 points)** Are there some data augmentation/regularization that authors have used? What are some techniques that could have been used and wasn't? (Go back to Lecture 7 or Chapter 7 of your book for more info.)

**1.5) (Bonus maximum 10 points)**. What other architectures would you try? For each family of models, please do a literature search and see if a paper on that architecture for the task of DNA binding detection has been 

# Question 2 - Literature Review for Sequence Labeling (20 points)

Read this paper: 


#### Mohammed AlQuraishi, "End-to-end differentiable learning of protein structure", 2018 bioRxiv 265231; doi: https://doi.org/10.1101/265231


We are interested in the task, the methods proposed in the paper, technical aspect of the implementation, and possible future work. 

After you read the article, go back to Figure 2. 

**2.1)( 5 points)** What is the architecture used in this task, to predict from the amino acid letter, to the three torsion angles? Describe the family of the architecture and few words on how the input sequence is converted to output in that architecture?


**2.2)(5 points)** Once the structure is predicted, what is the Loss function that is being optimized between the predicted structure and the ground truth structure? 

**2.3)(10 points) ** What are some alternative architectures that you would recommend as followup work? Name 2 potential architectures, and in a few sentences explain why the proposed model might work better.


**2.4) - no points just for your reference : The dataset for this paper is publicly available. Any new architecture that improves the prediction model will definitely be publication worthy and extremely valuable! Make a note of that and if you work on this topic, check the dataset out: https://github.com/aqlaboratory/proteinnet **

# Question 3 - Programming: Build Sequence Classifiers - Convolutional and Recurrent Neural Networks(60 points + 5 bonus points)

Let's build some models now, to try to classify each <a href="">protein</a> (represented as <a href="https://en.wikipedia.org/wiki/Protein_primary_structure">a sequence of amino acids</a>), into protein families.  

Why this is an important task? Briefly, our DNAs encode the code for proteins, which are molecular machines that make the cells work. 

![Our DNAs encode the code for proteins, which are molecular machines that make the cells work](https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Genetic_code.svg/580px-Genetic_code.svg.png) | ![Sequence to Structure](http://www.robotics.tu-berlin.de/fileadmin/_processed_/1/1f/csm_compbio_seq2struct_1614a2532b.jpg)

Given the sequence of the amino acids, there is great scientific value in being able to predict its 3D structure, and predict whether the protein will or will not bind to other chemical molecules such as drugs or other proteins. 
The applications are numerous in disease understanding and treatment (i.e. <a href="https://en.wikipedia.org/wiki/Amyloid_beta">Alzheimer's disease is related to *beta-amyloid* proteins in our brain not folding correctly and creating plaques</a>).

In this homework, we will focus on a dataset which has more than 400,000 protein sequences and their classes. The data and related pre-processing scriptes are is available <a href="https://www.kaggle.com/abharg16/predicting-protein-classification/data">here</a> and <a href="https://www.kaggle.com/abharg16/predicting-protein-classification/notebook">here</a>, which are super awesome.


Here, we will focus on predicting top few classes of proteins, from the sequence of the amino acids of that protein.
The data is available in the cluster in /scratch/nsr3/protein/rcsb/, although you're also welcome to have your own local copy of the data and work with that. We need two files: pdb_data_seq.csv and pdb_data_no_dups.csv

**Note: The content of your labs https://github.com/nyumc-dl/BMSC-GA-4493-Spring2019/blob/master/lab8/lab8_solutions.ipynb and https://github.com/nyumc-dl/BMSC-GA-4493-Spring2019/blob/master/lab7/lab7_solutions.ipynb will be directly useful for your answers **


** 3.1) (5 points)** Preprocessing. Most of the preprocessing is available in the kernel that came with the data. In paricular you can use the following to pre-process your data. Pre-process your data and tell us, 
how many data samples are available after the pre-processing? how many of the sequences are unique?

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Import Datasets
df_seq = pd.read_csv('/scratch/nsr3/protein/rcsb/pdb_data_seq.csv')
df_char = pd.read_csv('/scratch/nsr3/protein/rcsb/pdb_data_no_dups.csv')

# Filter for only proteins
protein_char = df_char[df_char.macromoleculeType == 'Protein']
protein_seq = df_seq[df_seq.macromoleculeType == 'Protein']

# Select only necessary variables to join
protein_char = protein_char[['structureId','classification']]
protein_seq = protein_seq[['structureId','sequence']]

model_f = protein_char.set_index('structureId').join(protein_seq.set_index('structureId'))
model_f = model_f.dropna()

** 3.2) (5 points) ** Select only the classes that have *more than 15,000 samples*. Only keep the rows that belong to one of these classes in your data. Which classes are there, and how many rows do you have after this filteration?

** 3.3) (5 points) ** Write a function, that takes a protein sequence *S* in, and converts it into a numpy array of size *25 x Len(S)*, which has the *one-hot encoding of the sequence*. 

You can use this list as all possible Amino Acid letters: **['H','V','G','A','P','C','D','I','R','E','K','L','W','T','Y','S','Q','F','N','M','U','X','Z','B','O']**

As an example, if S_0 is an 'H', the first column of our returned results has a 1.0 in row number 0 and, 0.0 in every other row. If it is a S_1 is a 'G', we put a 1.0 in row number 2 of that column, and a 0.0 in every other row in that column. We continue for all letterse in our sequence. 

**3.4) (5 points)** Now convert your data into train, test and validation set. Shuffle the rows, and split them with ratios of (train:60%, valid:20%, test:20%). 

(Hint: it's useful to set the random number seed before shuffling, so you get the same results over multiple runs).

**3.5) (5 points)** Now, convert your training, validation and test sequences to one-hot numpy arrays. 
Doing so in advance will save you computation time later. Also since we will be training a classifier, convert your one-hot label variables into the index. i.e. if your label is [0, 1, 0] convert it into [1]. If it is [0, 0, 1], convert it into [2]. (Hint: Use *numpy argmax* method if needed for fast implementation).

Write a dataloader similar to what we covered in the lab session (https://github.com/nyumc-dl/BMSC-GA-4493-Spring2019/blob/master/lab8/lab8_solutions.ipynb) so that we can begin to train our networks!

**3.6) (10 points) Now you are ready to build your sequence classification model! **

First, build a Convolutional sequence classification model similar to the architecture in question 1, (deepbind paper). 

Use Convolution, negative log likelihood (NLL) loss, and (optional: any additions to your architecture!), to go from the one-hot sequence of size *25 x len(S)* to multi-class classifier. 

At each epoch, compute **Average NLL loss** and **one AUC score per class** on both **train and validation set** 

Plot your validation and train loss over different epochs, and also print the AUCs on train and validation sets.

**3.7) (5 points + 5 Bonus points) ** The benefit of convolutional sequence model is that they are easier to interpret later. 
Use matplotlib and plt.imshow(), to visualize the filters of the *first layer convolution* that you have: 

(hint: an example, if the model is named model and the first layer of convolution is accessible via model.convnet1, the following code can give you those filters:
kernels = [k[0].data.numpy() for k in model.convnet1.weight])

**Note: It's ok if your model didn't converge at all. Just show the visualizations!**

**(Bonus 5 points):** Is there an equivalent of motif_plotter (i.e. line 31 in lab 8 https://github.com/nyumc-dl/BMSC-GA-4493-Spring2019/blob/master/lab8/lab8_solutions.ipynb) for Proteins and Amino Acids? Can you plot the convolution kernels using that library?

**3.8)(10 points)** Now, provide a second sequence classification model based on LSTMs. Build a simple LSTM model that takes as input the (25 x Len(s)) array, and ends with a softmax over total number of classees. (Hint - check your lab 7 session).

The rest of your experimental setting should be the same as section 3.6:

At each epoch, compute Average NLL loss and one AUC score per class on both train and validation set.

Plot your validation and train loss over different epochs, and also print the AUCs on train and validation sets.

**3.9) (5 points)** What are some other architectures that you could be using in future work? List a few and in a few sentences discuss why they might be a good fit for this task. 

**3.10)(5 points)** What are some other fine-tunning/regularizations/etc. that you could do in the future work, to improve the scores?